Boilerplate code below..

Setting a default seed for RNG

In [2]:
import random
import numpy as np

my_seed = 1337
random.seed(my_seed)
np.random.seed(my_seed)

In [3]:
import pandas as pd
import numpy as np
from typing import *
from IPython.display import display, HTML, Markdown

import warnings
warnings.filterwarnings('ignore')


def display_best_and_worse_recommendations(recommendations: pd.DataFrame):
    recommendations.sort_values('Estimated Prediction', ascending=False, inplace=True)

    top_recommendations = recommendations.iloc[:10]
    top_recommendations.columns = ['Prediction (sorted by best)', 'Movie Title']

    worse_recommendations = recommendations.iloc[-10:]
    worse_recommendations.columns = ['Prediction (sorted by worse)', 'Movie Title']

    display(HTML("<h1>Recommendations your user will love</h1>"))
    display(top_recommendations)

    display(HTML("<h1>Recommendations your user will hate</h1>"))
    display(worse_recommendations)
    

def load_movies_dataset() -> pd.DataFrame:
    movie_data_columns = [
    'movie_id', 'title', 'release_date', 'video_release_date', 'url',
    'unknown', 'Action', 'Adventure', 'Animation', "Children's",
    'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
    'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
    'War', 'Western'
    ]

    movie_data = pd.read_csv(
        'datasets/ml-100k/u.item', 
        sep = '|', 
        encoding = "ISO-8859-1", 
        header = None, 
        names = movie_data_columns,
        index_col = 'movie_id'
    )
    movie_data['release_date'] = pd.to_datetime(movie_data['release_date'])
    return movie_data

def load_ratings() -> pd.DataFrame:
    ratings_data = pd.read_csv(
        'datasets/ml-100k/u.data',
        sep = '\t',
        encoding = "ISO-8859-1",
        header = None,
        names=['user_id', 'movie_id', 'rating', 'timestamp']
    )
    return ratings_data[['user_id', 'movie_id', 'rating']]

def load_ratings_with_name() -> pd.DataFrame:
    ratings_data = load_ratings()
    movies_data = load_movies_dataset()
    ratings_data['user_id'] = ratings_data['user_id'].map(lambda k: f"User {k}")
    
    ratings_and_movies = ratings_data \
        .set_index('movie_id') \
        .join(movies_data['title']) \
        .reset_index()
    
    ratings_and_movies['movie_title'] = ratings_and_movies['title']
    return ratings_and_movies[['user_id', 'movie_title', 'rating']].sample(frac=1)
    
    


# A practical guide to Singular Value Decomposition in Python

Recommender systems have become increasingly popular in recent years, and are used by some of the largest websites in the world to predict the likelihood of a user taking an action on an item. In the world of Netflix, this means recommending similar movies to the ones you have seen. In the world of dating, this means suggesting matches similar to people you already showed interest in!

My path to recommenders has been an unusual one: from a Software Engineer to working on matching algorithms at a dating company, with a little background on machine learning. With my knowledge of Python and the use of basic SVD (Singular Value Decomposition) frameworks, I was able to understand SVDs from a practical standpoint of what you can do with them, instead of focusing on the science.

In my talk, you will learn 2 practical ways of generating recommendations using SVDs: matrix factorization and item similarity. We will be learning the high-level components of SVD the "doer way": we will be implementing a simple movie recommendation engine with the help of Jupiter notebooks, the MovieLens database, and the Surprise recommendation package.

 - Recommendations via Matrix Factorization: Performing predict() manually

## Table of contents

 - Downloading and exploring the MovieLens dataset
 - Training a SVD using Surprise in 4 simple steps
 - Using the predict() API inside of Surprise
 - recommendations via Product based CF: Finding similarity between vectors

# Downloading and exploring the MovieLens dataset

<p><img src="https://static1.squarespace.com/static/51cdafc4e4b09eb676a64e68/t/579282fabebafbb6c366252c/1469219594863/" alt="Drawing" style="width: 400px; float: left"/></p>


# Exploring the Dataset



- Open Source dataset
- 20 million ratings
- 27,000 movies
- 138,000 users

In [4]:
movie_data = load_movies_dataset()

In [5]:
ratings_data = load_ratings_with_name()
ratings_data.head(5)

,user_id,movie_title,rating
36649,User 742,Jerry Maguire (1996),4
2478,User 908,"Usual Suspects, The (1995)",3
82838,User 758,Real Genius (1985),4
69729,User 393,Things to Do in Denver when You're Dead (1995),3
36560,User 66,Jerry Maguire (1996),4


# Training a SVD using Surprise in 4 simple steps

Let's take the **interactions** between the Users and Movies, and generate **latent features**  

In [8]:
from scipy.spatial.distance import cosine
import numpy as np


def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
    return cosine(vector_a, vector_b)

def get_vector_by_movie_title(movie_title: str, trained_model: SVD) -> np.array:
    movie_row_idx = trained_model.trainset._raw2inner_id_items[movie_title]
    return trained_model.qi[movie_row_idx]

In [18]:
from surprise import SVD, NMF, accuracy
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, train_test_split

# Step 1: create a Reader.
# A reader tells our SVD what the lower and upper bound of our ratings is.
# MovieLens ratings are from 1 to 5
reader = Reader(rating_scale=(1, 5))

In [19]:
# Step 2: create a new Dataset instance with a DataFrame and the reader
# The DataFrame needs to have 3 columns in this specific order: [user_id, product_id, rating]
data = Dataset.load_from_df(ratings_data, reader)

In [7]:
# Step 3: keep 25% of your trainset for testing
trainset, testset = train_test_split(data, test_size=.25)

In [20]:
trainset, testset = train_test_split(data, test_size=.01)

In [29]:
# Step 4: train a new SVD
model = SVD(n_factors=100, biased=False)
model.fit(trainset)

# Optionally, validate the RMSE (root-mean-square error) to ensure model training was effectiv 
predictions = model.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9665


0.9664994793815466

In [30]:
ratings_data

,user_id,movie_title,rating
36649,User 742,Jerry Maguire (1996),4
2478,User 908,"Usual Suspects, The (1995)",3
82838,User 758,Real Genius (1985),4
69729,User 393,Things to Do in Denver when You're Dead (1995),3
36560,User 66,Jerry Maguire (1996),4
86954,User 435,"Glimmer Man, The (1996)",2
81731,User 314,Philadelphia (1993),5
50694,User 925,Crash (1996),4
87092,User 269,That Thing You Do! (1996),1
77160,User 566,Fried Green Tomatoes (1991),4


In [31]:
toy_story_vector: np.array = get_vector_by_movie_title('Monty Python and the Holy Grail (1974)', model)


similarity_table = []
for movie_title in model.trainset._raw2inner_id_items.keys():
    movie_vector = get_vector_by_movie_title(movie_title, model)
    similarity_score = cosine_distance(toy_story_vector, movie_vector)
    similarity_table.append((similarity_score, movie_title))

pd.DataFrame(
    similarity_table,
    columns=['similarity', 'movie title']
).sort_values('similarity', ascending=True).head(10)

,similarity,movie title
35,0.000000,Monty Python and the Holy Grail (1974)
790,0.221785,Delicatessen (1991)
77,0.236081,Monty Python's Life of Brian (1979)
836,0.253139,Fresh (1994)
932,0.255325,"Sweet Hereafter, The (1997)"
570,0.257154,Swimming with Sharks (1995)
416,0.258974,Once Upon a Time in the West (1969)
315,0.259394,Bob Roberts (1992)
199,0.261901,"Maltese Falcon, The (1941)"
357,0.264360,"Princess Bride, The (1987)"


# Predict, under the hood

So far we have seen how the `predict()` API works in surface. But how does it **really** work inside of surprise. It's, surprisingly, simple! (get the pun?)

But before we go there, let's go back to our Feature Vectors

![Latent Features](https://cdn-images-1.medium.com/max/1600/0*_gKhyxIC3wup0cCE.jpg)

## Looking at the Movie matrix (vT)

Let's take a look at the latent features for every movie. Product features can be found in the `qi` attribute.
 - create a DataFrame that maps product matrix row index to movie
 - join the newly created dataframe with the movie dataset
 - join the newly created dataframe with the latent features